# Time Series Project: Amazon Stock Data

Coded by Luna McBride

This is a project meant to mess with time series data. I will be using the opening rate as the key metric, as it is consistent. The below sources are ones I am using to get an idea of the functions and concepts behind the practice. 

Sources: https://www.kaggle.com/gayatry/population-prediction-ar-vs-arima , https://www.youtube.com/watch?v=e8Yw4alG16Q

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt #Plotting
%matplotlib inline

import warnings #What to do with warnings
warnings.filterwarnings("ignore") #Ignore the warnings

plt.rcParams["figure.figsize"] = (10,10) #Make the plots bigger by default
plt.rcParams["lines.linewidth"] = 2 #Setting the default line width
plt.style.use("ggplot") #Define the style of the plot

from statsmodels.tsa.seasonal import seasonal_decompose #Describes the time data
from statsmodels.tsa.stattools import adfuller #Check if data is stationary
from statsmodels.graphics.tsaplots import plot_acf #Compute lag for ARIMA
from statsmodels.graphics.tsaplots import plot_pacf #Compute partial lag for ARIMA
from statsmodels.tsa.arima_model import ARIMA #Predictions and Forecasting

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
amazon = pd.read_csv("../input/amazon-stock-price-1997-to-2020/Amazon.csv") #Get our stock data from the CSV
amazon.head(10) #Take a peek at the Amazon data

---

# Check for Null Values

In [ ]:
print(amazon.isnull().any()) #Check for null values

There are no null values.

---

# Create Subset for Values I Want

In [ ]:
amazonOpen = amazon[["Date", "Open"]].copy() #Get the date and open columns
amazonOpen["Date"] = pd.to_datetime(amazonOpen["Date"]) #Ensure the date data is in datetime format
amazonOpen.set_index("Date", inplace = True) #Set the date to the index
amazonOpen = amazonOpen.asfreq("b") #Set the frequency
amazonOpen = amazonOpen.fillna(method  = "bfill") #Fill null values with future values

#amazonOpen.index #Make sure the frequency remains intact
amazonOpen.head(12) #Take a peek at the open data

In [ ]:
y = amazonOpen.plot(title = "Amazon Stocks (Open)") #Get an idea of the data
y.set(ylabel = "Price at Open") #Set the y label to open
plt.show() #Show the plot

---

# Check Trend/Seasonality

In [ ]:
decomp = seasonal_decompose(amazonOpen, model = "multiplicative") #Decompose the data
x = decomp.plot() #Plot the decomposed data

The seasonal looks like a red blob, but the trend is positive.

---

# Check if Data is Stationary

In [ ]:
print("ADFuller Test; Significance: 0.05") #Print the significance level
adf = adfuller(amazonOpen["Open"]) #Call adfuller to test
print("ADF test static is {}".format(adf[1])) #Print the adfuller results

ADFuller gave a clean 1.0. That means this data is not stationary, even for huge significance values.

---

# Make the Data Stationary

In [ ]:
openLog = np.log(amazonOpen) #Take the log of the set for normalization
openStationary = openLog - openLog.shift() #Get a stationary set by subtracting the shifted set
openStationary = openStationary.dropna() #Drop generated null values from the set
openStationary.plot(title = "Stationary Amazon Stocks") #Plot the stationary set

In [ ]:
print("ADFuller Test; Significance: 0.05") #Print the significance level
adf = adfuller(openStationary["Open"]) #Call adfuller to test
print("ADF test static is {}".format(adf[1])) #Print the adfuller results

ADFuller is now below significance. The data is now stationary.

In [ ]:
decomp = seasonal_decompose(openStationary) #Decompose the stationary data
x = decomp.plot() #Plot the decomposition

---

# Preparation for ARIMA

## Differencing Term D

In [ ]:
fig,axes = plt.subplots(2,2) #Set a subset for the data visualizations

a = axes[0,0].plot(amazonOpen["Open"]) #Plot the original data
a = axes[0,0].set_title("Original Data") #Give the original data a name
b = plot_acf(amazonOpen["Open"],ax=axes[0,1]) #Plot the ACF of the original data

x = axes[1,0].plot(openStationary["Open"]) #Plot the stationary data
x = axes[1,0].set_title("Stationary Data") #Give the stationary data a name
y = plot_acf(openStationary["Open"],ax=axes[1,1]) #Plot the ACF of the stationary data

The Stationary data appears to be mostly in the fair range, with some variance due to the large jump in the data. 1 appears to be a good number for D.

## AR Lag (P)

In [ ]:
fig,axes = plt.subplots(1,2) #Create a subplot for the Partial ACF

a = axes[0].plot(openStationary["Open"]) #Plot the stationary data
a = axes[0].set_title("Stationary") #Ensure the stationary data is named
b = plot_pacf(openStationary["Open"], ax = axes[1], method = "ols") #Plot the partial ACF

5 appears to be a good number for P.

## MA Lag (Q)

In [ ]:
fig,axes = plt.subplots(1,2) #Create a subplot for the ACF

a = axes[0].plot(openStationary["Open"])#Plot the stationary data
a = axes[0].set_title("Stationary") #Ensure the stationary data is named
b = plot_acf(openStationary["Open"], ax = axes[1]) #Plot the ACF

5 appears to be a good number for the Q metric as well.

---

# Train the ARIMA

In [ ]:
model = ARIMA(openStationary, order = (5, 1, 5)) #Build the ARIMA model
fitModel = model.fit(disp = 1) #Fit the ARIMA model

---

# Predict with ARIMA

In [ ]:
plt.rcParams.update({"figure.figsize" : (12,6), "lines.linewidth" : 0.05, "figure.dpi" : 100}) #Fix the look of the graph, dimming it to show the red

x = fitModel.plot_predict(dynamic = False) #Fit the ARIMA model
x = plt.title("Forecast Fitting") #Add a stock title
plt.show() #Show the ARIMA plot

---

# Validation

In [ ]:
plt.rcParams.update({"figure.figsize" : (12,5), "lines.linewidth": 2}) #Fix the line width
length = int((len(amazonOpen)*9)/10) #Get 9/10 of the length of the data
print(length) #Print the length to make sure it actually is an int

In [ ]:
train = amazonOpen[:length] #Use 9/10 of the data for the train set
test = amazonOpen[length:] #Use the rest for testing
modelValid = ARIMA(train,order=(5,1,5)) #Create a model for the train set
fitModelValid = modelValid.fit(disp= -1) #Fit the model

In [ ]:
fc,se,conf = fitModelValid.forecast(len(amazonOpen) - length) #Forcast over the test area
forecast = pd.Series(fc, index = test.index) #Get the forecast for the area

In [ ]:
#Add labels for the train, test, and forecast
plt.plot(train,label = "Training Data") 
plt.plot(test,label = "Actual Continuation")
plt.plot(forecast,label = "Forecasted Continuation", color = "g")

plt.title("ARIMA Forecast") #Add the Forecast title
plt.legend(loc = "upper left") #Put the legend in the top left
plt.xlabel("Year") #Add the year label to the bottom
plt.ylabel("Open Price") #Add the open price to the y axis

Putting 1/10 of the test data for the stocks appears to show a fair prediction for the open price. It had to go far enough to account for the massive growth in the late 2010's. It does not account for the massive jump in 2020, but 2020 is an anomoly for Amazon stocks, given Covid.

---

# Predicting

In [ ]:
modelPred = ARIMA(amazonOpen,order=(5,1,5)) #Create a model for the whole data
fitModelPred = modelPred.fit(disp= -1) #Fit the model

In [ ]:
fitModelPred.plot_predict(1,len(amazonOpen) + 1000) #Plot predictions for the next thousand days
x = fitModelPred.forecast(1000) #Forecast the prediction for the next thousand days.
x = plt.title("Amazon Stock Forecast") #Add a stock title
x = plt.xlabel("Year") #Add the year label to the bottom
x = plt.ylabel("Open Price") #Add the open price to the y axis

The model predicts an overall increase in stock price. The slope of the prediction is more akin to how level the majority of the graph is, but the confidence interval is wide to account for variance. In this case, the spike caused by Covid is likely the big indicator for why in the interval is so large. In general, it is fair to assume Amazon stocks will continue to rise given previous trends, but we need to keep in mind we are coming out of a pandemic. We will likely continue to feel its wrath linger long after it is handled, which could cause unexpected spikes or dips from people getting back to life and deciding whether or not to continue using Amazon services as much as they have. At the current moment, however, this feels like a fair prediction.